# Conventional time series model

## MA(q)
$$X_t = Z_t + \sum_{k=1}^{q}\theta_kZ_{t-k}$$

where $$Z_t \sim N(0, \sigma^{2})$$ 

For example

$MA(2)$:
$$X_t = Z_t + \theta_1Z_{t-1} + \theta_2Z_{t-2} $$

In [ ]:
import numpy as np

Z = np.random.randn(1000)

In [ ]:
theta_1 = 0.7
theta_2 = 0.2

X = [Z[i] + theta_1 * Z[i-1] + theta_2 * Z[i-2] for i in range(2, len(Z))]

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 6))

ax.plot(X)

In [ ]:
# compute acf

from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(np.array(X)) # plot_acf only accepts np.ndarray type 

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Autoregressive Integrated Moving Average ARIMA(p,d,q) Model, p for autoregression, q for moving average, d for difference

model = ARIMA(X, order=(0, 0, 2))

In [ ]:
result = model.fit()

In [ ]:
result.summary()  

## AR(p)
$$X_t = Z_t + \sum_{k=1}^{p}\theta_kX_{t-k}$$

where $$Z_t \sim N(0, \sigma^{2})$$ 

For example 
AR(2):
$$X_t = Z_t + \theta_1X_{t-1} + \theta_2X_{t-2}$$

Random walk is an AR(1) process:

$$X_t = Z_t + X_{t-1}$$

In [ ]:
# Random walk example

Z = np.random.randn(1000)

X = [Z[0]]

for z in Z[1:]:
    X.append(X[-1] + z)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

ax.plot(X)

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

plot_pacf(np.array(X))

In [ ]:
model = ARIMA(X, order=(1, 0, 0))
result = model.fit()
result.summary()  

In [ ]:
# AR(2)

theta_1 = 0.4
theta_2 = -0.2

X = [Z[0], Z[1] + theta_1 * Z[0]]

for z in Z[2:]:
    X.append(z + theta_1 * X[-1] + theta_2 * X[-2])

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

ax.plot(X)

In [ ]:
plot_pacf(np.array(X))

In [ ]:
model = ARIMA(X, order=(2, 0, 0))
result = model.fit()
result.summary()  

# Real world data

In [ ]:
import os

import pandas as pd

os.chdir("/".join(os.getcwd().split("/")[:-1]))


def load_data():
    
    filename = "data/raw/原始資料.xlsx"

    xls = pd.ExcelFile(filename, engine='openpyxl')
    df = pd.read_excel(xls, index_col=0)

    df = df[['代號', '數量']]

    df.reset_index(inplace=True)
    df_sum = df.groupby(['代號', '交易日期'], as_index=False)['數量'].sum()

    # Identify unique 代號
    unique_product_code = np.unique(df_sum['代號'])

    product_df_list = []

    # Replace all the 數量 with the 代號
    for code in unique_product_code:
        temp = df_sum[df_sum['代號']==code].set_index("交易日期")
        temp.drop(labels=['代號'], inplace=True, axis=1)
        temp.rename(columns={'數量': code}, inplace=True)
        product_df_list.append(temp)

    df_parsed = pd.concat(product_df_list, axis=1)
    
    return df_parsed

df_parsed = load_data()

In [ ]:
df_parsed

In [ ]:
df_A = df_parsed[['A']]
df_A = df_A.resample('1D').sum()
df_A.fillna(0, inplace=True)

weeks = [idx.week for idx in df_A.index]
df_A['week'] = weeks
years = [idx.year for idx in df_A.index]
df_A['year'] = years

df_A_year_week = df_A.groupby(['week', 'year'], as_index=False)['A'].sum()

df_A_year_week.sort_values(by=['year', 'week'], inplace=True)

df_A_year_week.reset_index(drop=True, inplace=True)

In [ ]:
df_A_year_week.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

ax.plot(df_A_year_week['A'])

In [ ]:
A = df_A_year_week['A'].values

## MA(q)

In [ ]:
plot_acf(A)

In [ ]:
model_MA5 = ARIMA(A, order=(0, 0, 5))
result = model_MA5.fit()
result.summary()

In [ ]:
model_MA3 = ARIMA(A, order=(0, 0, 3))
result = model_MA3.fit()
result.summary()

## How to get an 'optimized' model with respect to AIC by tuning q

In [ ]:
max_aic = np.inf
optimized_q = 0

for q in range(1, 12):
    
    model = ARIMA(A, order=(0, 0, q))
    fitted_model = model.fit()
    aic = fitted_model.aic
    
    if aic < max_aic:
        max_aic = aic
        optimized_q = q

In [ ]:
max_aic

In [ ]:
optimized_q

## MA(q) Prediction

In [ ]:
predictions = result.predict(start=0, end=len(A)-1)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

ax.plot(df_A_year_week['A'], c='b', label='Obervation')
ax.plot(predictions, c='r', label='Prediction')

In [ ]:
result.forecast(steps=10)

## AR(p)

In [ ]:
plot_pacf(A)

In [ ]:
model_AR2 = ARIMA(A, order=(2, 0, 0))
result = model_AR2.fit()
result.summary()

## ARMA(p, q)
$$X_t = Z_t + \sum_{k=1}^{p}\theta_kX_{t-k} + \sum_{k=1}^{q}\phi_kZ_{t-k}$$

## Backward shift operator

$$ BX_t = X_{t-1} $$
$$ B^2X_t = X_{t-2} $$
$$ B^kX_t = X_{t-k} $$


With backward shift operator, for formula can be simplified:

$$(1 -  \sum_{k=1}^{p}\theta_kB^k)X_t = (1 + \sum_{k=1}^{q}\phi_kB^k)Z_t$$

In [ ]:
model_ARMA = ARIMA(A, order=(2, 0, 4))
result = model_ARMA.fit()
result.summary()

In [ ]:
# Get optimized p and q for ARMA

max_aic = np.inf
optimized_q = 0
optimized_p = 0

for q in range(1, 6):
    for p in range(1, 4):
        model = ARIMA(A, order=(p, 0, q))
        fitted_model = model.fit()
        aic = fitted_model.aic
        
#         print(f"aic = {aic}, p = {p}, q = {q}")
        
        if aic < max_aic:
            max_aic = aic
            optimized_q = q
            optimized_p = p

In [ ]:
model = ARIMA(A, order=(1, 0, 1))
fitted_model = model.fit()
predictions = fitted_model.predict(start=0, end=len(A)-1)

fig, ax = plt.subplots(figsize=(8,6))

ax.plot(df_A_year_week['A'], c='b', label='Obervation')
ax.plot(predictions, c='r', label='Prediction')

## ARIMA

ARIMA(p, d, q) process:
    A process $X_t$ is Autoregressive Interated Moving Average of order (p, q, d) if 
    
$$Y_t:=\nabla^dX_t = (1-B)^dX_t$$  

is ARMA(p, q). 

$$Y_t \sim ARMA(p, q)$$ => $$X_t \sim ARIMA(p, d, q)$$


1. $d = 1$ or $d = 2$
2. ACF decays very slowly.

In [ ]:
A_diff = np.diff(A)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

ax.plot(A_diff)

In [ ]:
plot_acf(A_diff)

In [ ]:
model_ma1_i = ARIMA(A_diff, order=(0, 0, 1))
fitted_model = model_ma1_i.fit()
fitted_model.summary()

In [ ]:
plot_pacf(A_diff)

In [ ]:
model_ap4_i = ARIMA(A_diff, order=(4, 0, 0))
fitted_model = model_ap4_i.fit()
fitted_model.summary()

In [ ]:
# ARMA

max_aic = np.inf
optimized_q = 0
optimized_p = 0

for q in range(1, 3):
    for p in range(0, 5):
        model = ARIMA(A_diff, order=(p, 0, q))
        fitted_model = model.fit()
        aic = fitted_model.aic
        
        if aic < max_aic:
            max_aic = aic
            optimized_q = q
            optimized_p = p

In [ ]:
max_aic

In [ ]:
optimized_p

In [ ]:
optimized_q

# Seasonal ARIMA (SARIMA)

Box - Jenkins Seasonal ARIMA model

\* Data might contain seasonal periodic component in addition to correlation with recent lags.

\* It repeats every $s$ observations.

\* For a time series of monthly observations, $X_t$ might depend on annual lags. i.e. $X_{t-12}$, $X_{t-24}$ 

\* Quarterly data might have period of $s=4$

## Pure Seasonal ARMA process

$ARMA(P, Q)$ has the form:
$$\Phi_p(B^s)X_t = \Theta_Q(B^s)Z_t$$
where
$$\Phi_P(B^s) = 1 - \Phi_1B^s - \Phi_2B^{2s} - \cdots - \Phi_PB^(Ps)$$
and
$$\Theta_Q(B^s) = 1 + \Theta_1B^s + \Theta_2B^{2s} + \cdots + \Theta_QB^(Qs)$$

Example 1: Seasonal $ARMA(1,0)_{12}$ has the form
$$(1-\Phi_1B^{12})X_t = Z_t$$
i.e.
$$X_t=\Phi_1X_{t-12} + Z_t$$

Example 2: Seasonal $ARMA(1,1)_{12}$ has the form
$$(1-\Phi_1B^{12})X_t = (1+\Theta_1B^{12})Z_t$$
i.e.
$$X_t=\Phi_1X_{t-12} + Z_t + \Theta_1Z_{t-12}$$


## Seasonal ARIMA process (SARIMA)

$SARIMA(p,d,q, P,D,Q)_s$ has the form

$$\Phi_P(B^s)\phi_p(B)(1-B^s)^D(1-B)^dX_t = \Theta_Q(B^s)\theta_q(B)Z_t$$

where

$$\theta_q(B) = 1 + \theta_1B + \cdots +\theta_qB^q$$

$$\Theta_Q(B^s) = 1 + \Theta_1B^s + \Theta_2B^{2s} + \cdots + \Theta_QB^{Qs}$$

$$\phi_p(B) = 1 - \phi_1B - \phi_2B^2 - \cdots - \phi_pB^p$$

$$\Phi_P(B^s) = 1 - \Phi_1B^s - \Phi_2B^{2s} - \cdots - \Phi_PB^{Ps}$$

Example 3 - $SARIMA(1,0,0,1,0,1)_{12}$

$$(1-\phi_1B)(1-\Phi_1B^{12})X_t = (1+\Phi_1B^{12})Z_t$$

$$(1 - \phi_1B - \Phi_1B^{12} + \phi_1\Phi_1B^{13})X_t = Z_t + \Phi_1Z_{t-12}$$

Thus

$$X_t = \phi_1X_{t-1} + \Phi_1X_{t-12} - \phi_1\Phi_1X_{t-13} + Z_t + \Phi_1Z_{t-12}$$

Example 4 - $SARIMA(0,1,1,0,0,1)_4$

$$(1-B)X_t = (1+\Theta_1B^4)(1+\theta_1B)Z_t$$

## ACF of SARIMA models

Example: $SARIMA(0,0,1,0,0,1)_{12}$

$$X_t=(1+\Theta_1B^{12})(1+\theta_1B)Z_t$$

$\theta=0.7$ and $\Theta=0.6$ (ignore the subscription).

$$X_t=Z_t+0.7Z_{t-1}+0.6Z_{t-12}+0.42Z_{t-13}$$

In [ ]:
Z = np.random.randn(1000)

X = [Z[i] + 0.7 * Z[i-1] + 0.6 * Z[i-12] + 0.42 * Z[i-13] for i in range(13, 1000)]


In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

ax.plot(X)

In [ ]:
plot_acf(np.array(X))

## Let us look at the real world data

In [ ]:
_ = plot_acf(A)

In [ ]:
_ = plot_pacf(A)

In [ ]:
_ = plot_acf(A_diff)

In [ ]:
_ = plot_pacf(A_diff)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(A, order=(1,1,1), seasonal_order=(1,0,0,7))
results = model.fit()

In [ ]:
results.summary()

In [ ]:
model = SARIMAX(A, order=(2,1,3), seasonal_order=(0,1,1,7), )
results = model.fit()
results.summary()

In [ ]:
pred_A = results.predict(start=0, end=len(A)-1)

In [ ]:
A_residue = A - pred_A

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

ax.plot(A_residue)

In [ ]:
_ = plot_acf(A_residue)

In [ ]:
_ = plot_pacf(A_residue)

In [ ]:
# 

fig, ax = plt.subplots(figsize=(16, 6))

ax.plot(A, c='b', label='ground_truth')
ax.plot(pred_A, c='r', label='prediction')

ax.legend()